In [1]:
#  источники - https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook
# https://spacy.io/models/ru#ru_core_news_md

import re  # Для препроцессинга
!pip install pandas
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
# Считываю файл
data = pd.read_csv('Сlean.csv')
df=data['description']

In [ ]:
df.isnull().sum()

In [ ]:
# Загружаю модель
!python -m spacy download ru_core_news_sm

In [ ]:
nlp = spacy.load('ru_core_news_sm', disable=['ner', 'parser']) # отключение распознавания именованных объектов для повышения скорости
def cleaning(doc):
    #Лемматизируется и удаляет стоп-слова
    # doc должен быть пространственным объектом Doc
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec использует контекстные слова для изучения векторного представления целевого слова,
    # если предложение состоит всего из одного или двух слов,
    # польза от тренинга мала, поэтому
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
    #Удаляю неалфавитные символы:
brief_cleaning = (re.sub("[^А-Яа-я]", ' ', str(row)).lower() for row in data['description'])

In [ ]:
# Атрибут space .pipe() для ускорения процесса очистки:
# Подбираю оптимальное batch_size=2000, n_process=4
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=2000, n_process=4)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
#Помещаю результаты в фрейм данных, чтобы удалить недостающие значения и дубликаты:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()

In [ ]:
#Удаляю пробелы которых больше или равно 2 и сохраняю данные
df_clean['clean']= df_clean['clean'].str.replace(' {2,}', ' ', regex=True)
df_clean.to_csv('PRE.csv',index=True)